##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TFX Pipeline と TensorFlow Data Validation を使ったデータ検証

注意: このチュートリアルを Colab ノートブックで実行することをお勧めします。セットアップは必要ありません。「Google Colab で実行」をクリックしてください。

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/penguin_tfdv"> <img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/tfx/tutorials/tfx/penguin_tfdv.ipynb"> <img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/tfx/tutorials/tfx/penguin_tfdv.ipynb"> <img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/tfx/tutorials/tfx/penguin_tfdv.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table></div>

ノートブックを使用したこのチュートリアルでは、入力データを検証して ML モデルを作成する TFX パイプラインを作成して実行します。このノートブックは、[単純な TFX パイプラインのチュートリアル](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple)で構築した TFX パイプラインを基盤として使用しています。そのチュートリアルにまだ目を通していない場合は、このノートブックを進める前に読むことをお勧めします。

データサイエンスや機械学習プロジェクトの最初のタスクは、データを理解しクリーンアップすることです。これには、以下が含まれます。

- 各特徴量に関するデータ型や分布などの情報（平均値、一意の数など）を理解する
- データを説明する暫定的なスキーマを生成する
- 特定のスキーマに関するデータの異常値と欠損値を特定する

このチュートリアルでは、2 つの TFX パイプラインを作成します。

まず、データセットを分析して、そのデータセットの暫定的なスキーマを生成するパイプラインを作成します。このパイプラインには、`StatisticsGen` と `SchemaGen` という 2 つの新しいコンポーネントが含められます。

データに適切なスキーマができたら、前のチュートリアルのパイプラインをベースに、機械学習分類モデルをトレーニングするパイプラインを作成します。このパイプラインでは、最初のパイプラインのスキーマと、入力データを検証する新しい `ExampleValidator` コンポーネントを使用します。

StatisticsGen、SchemaGen、および ExampleValidator という 3 つの新しいコンポーネントは、データ分析と検証用の TFX コンポーネントで、[TensorFlow Data Validation](https://www.tensorflow.org/tfx/guide/tfdv) ライブラリを使って実装されます。

TFX の様々な概念についての詳細は、[TFX パイプラインの理解](https://www.tensorflow.org/tfx/guide/understanding_tfx_pipelines)をご覧ください。

## セットアップ

まず、TFX Python パッケージをインストールし、モデルに使用するデータセットをダウンロードする必要があります。

### Pip のアップグレード

ローカルで実行する場合にシステムの Pip をアップグレードしないように、Colab で実行していることを確認してください。もちろん、ローカルシステムは個別にアップふれーどできます。

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### TFX をインストールする


In [ ]:
!pip install -U tfx

### shapely をアンインストールする

TODO(b/263441833) これは、InportError を回避するための一時的な策です。最終的には、追加の依存関係をアンインストールする代わりに Bigquery の最近のバージョンをサポートすることで処理される必要があります。

In [ ]:
!pip uninstall shapely -y

### ランタイムを再起動しましたか？

Google Colab を使用している場合は、上記のセルを初めて実行した後に、「RESTART RUNTIME」ボタンをクリックするか、「Runtime」&gt;「Restart runtime ...」メニューを使って、ランタイムを再起動する必要があります。これは、Colab がパッケージを読み込むために必要な作業です。

TensorFlow と TFX のバージョンを確認します。

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

### 変数をセットアップする

パイプラインを定義するために使用する変数がいくつかあります。これらの変数は、必要に応じてカスタマイズすることが可能です。デフォルトでは、パイプラインからのすべての出力は、現在のディレクトリに生成されます。

In [ ]:
import os

# We will create two pipelines. One for schema generation and one for training.
SCHEMA_PIPELINE_NAME = "penguin-tfdv-schema"
PIPELINE_NAME = "penguin-tfdv"

# Output directory to store artifacts generated from the pipeline.
SCHEMA_PIPELINE_ROOT = os.path.join('pipelines', SCHEMA_PIPELINE_NAME)
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
# Path to a SQLite DB file to use as an MLMD storage.
SCHEMA_METADATA_PATH = os.path.join('metadata', SCHEMA_PIPELINE_NAME,
                                    'metadata.db')
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

### サンプルデータを準備する

チュートリアルの TFX パイプラインで使用するサンプルデータセットをダウンロードします。使用するデータセットは [Palmer Penguins データセット](https://allisonhorst.github.io/palmerpenguins/articles/intro.html)で、他の [TFX の例](https://github.com/tensorflow/tfx/tree/master/tfx/examples/penguin)でも使用されているデータセットです。

このデータセットには、4 つの数値特徴量があります。

- culmen_length_mm
- culmen_depth_mm
- flipper_length_mm
- body_mass_g

すべての特徴量はすでに範囲 [0,1] を持つように正規化されています。ペンギンの `species`（種）を予測する分類モデルを構築します。

TFX ExampleGen コンポーネントはディレクトリから入力を読み取るため、ディレクトリを作成してデータセットをそれにコピーする必要があります。

In [ ]:
import urllib.request
import tempfile

DATA_ROOT = tempfile.mkdtemp(prefix='tfx-data')  # Create a temporary directory.
_data_url = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/penguin/data/labelled/penguins_processed.csv'
_data_filepath = os.path.join(DATA_ROOT, "data.csv")
urllib.request.urlretrieve(_data_url, _data_filepath)

CSV ファイルを見てみましょう。

In [ ]:
!head {_data_filepath}

5 つの特徴量カラムがあります。`species` は 0、1、または 2 のいずれかで、その他すべての特徴量には 0 と 1 の間の値があります。このデータセットを分析する TFX パイプラインを作成しましょう。

## 暫定的なスキーマを生成する

TFX パイプラインは Python API で定義されます。入力 Example から自動的にスキーマを生成するパイプラインを作成します。このスキーマは、人間がレビューして必要に応じて調整することが可能です。スキーマが出来上がったら、後のタスクでトレーニングと Example の検証に使用できます。

[単純な TFX パイプラインのチュートリアル](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple)で使用される `CsvExampleGen` の他に、`StatisticsGen` と `SchemaGen` を使用します。

- [StatisticsGen](https://www.tensorflow.org/tfx/guide/statsgen) はデータセットの統計を計算します。
- [SchemaGen](https://www.tensorflow.org/tfx/guide/schemagen) は統計を調べ、最初のデータスキーマを作成します。

これらのコンポーネントの詳細については、各コンポーネントのガイドか [TFX コンポーネントのチュートリアル](https://www.tensorflow.org/tfx/tutorials/tfx/components_keras)をご覧ください。

### パイプライン定義を記述する

TFX パイプラインを作成する関数を定義します。`Pipeline` オブジェクトは、TFX がサポートするパイプラインオーケストレーションシステムを使って実行できる TFX パイプラインです。

In [ ]:
def _create_schema_pipeline(pipeline_name: str,
                            pipeline_root: str,
                            data_root: str,
                            metadata_path: str) -> tfx.dsl.Pipeline:
  """Creates a pipeline for schema generation."""
  # Brings data into the pipeline.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # NEW: Computes statistics over data for visualization and schema generation.
  statistics_gen = tfx.components.StatisticsGen(
      examples=example_gen.outputs['examples'])

  # NEW: Generates schema based on the generated statistics.
  schema_gen = tfx.components.SchemaGen(
      statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)

  components = [
      example_gen,
      statistics_gen,
      schema_gen,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

### パイプラインを実行する

前のチュートリアルのように、`LocalDagRunner` を使用します。

In [ ]:
tfx.orchestration.LocalDagRunner().run(
  _create_schema_pipeline(
      pipeline_name=SCHEMA_PIPELINE_NAME,
      pipeline_root=SCHEMA_PIPELINE_ROOT,
      data_root=DATA_ROOT,
      metadata_path=SCHEMA_METADATA_PATH))

パイプラインが正常に完了すると、「INFO:absl:Component SchemaGen is finished.」が表示されます。

データセットを理解するために、パイプラインの出力を調べます。

### パイプラインの出力をレビューする

前のチュートリアルで説明した通り、TFX パイプラインは、アーティファクトと、アーティファクトのメタデータとパイプラインの実行を含む [metadata DB(MLMD)](https://www.tensorflow.org/tfx/guide/mlmd) の 2 種類の出力を生成します。これらの出力の場所は、上のセルで定義されています。デフォルトでは、アーティファクトは `pipelines` ディレクトリに、メタデータは sqlite データセットとして `metadata` ディレクトリに保存されます。

MLMD API を使用して、プログラムでこれらの出力の場所を特定することができます。まず、生成されたばかりの出力アーティファクトを検索するユーティリティ関数を定義しましょう。


In [ ]:
from ml_metadata.proto import metadata_store_pb2
# Non-public APIs, just for showcase.
from tfx.orchestration.portable.mlmd import execution_lib

# TODO(b/171447278): Move these functions into the TFX library.

def get_latest_artifacts(metadata, pipeline_name, component_id):
  """Output artifacts of the latest run of the component."""
  context = metadata.store.get_context_by_type_and_name(
      'node', f'{pipeline_name}.{component_id}')
  executions = metadata.store.get_executions_by_context(context.id)
  latest_execution = max(executions,
                         key=lambda e:e.last_update_time_since_epoch)
  return execution_lib.get_output_artifacts(metadata, latest_execution.id)

# Non-public APIs, just for showcase.
from tfx.orchestration.experimental.interactive import visualizations

def visualize_artifacts(artifacts):
  """Visualizes artifacts using standard visualization modules."""
  for artifact in artifacts:
    visualization = visualizations.get_registry().get_visualization(
        artifact.type_name)
    if visualization:
      visualization.display(artifact)

from tfx.orchestration.experimental.interactive import standard_visualizations
standard_visualizations.register_standard_visualizations()

次に、パイプライン実行の出力を調べることができます。

In [ ]:
# Non-public APIs, just for showcase.
from tfx.orchestration.metadata import Metadata
from tfx.types import standard_component_specs

metadata_connection_config = tfx.orchestration.metadata.sqlite_metadata_connection_config(
    SCHEMA_METADATA_PATH)

with Metadata(metadata_connection_config) as metadata_handler:
  # Find output artifacts from MLMD.
  stat_gen_output = get_latest_artifacts(metadata_handler, SCHEMA_PIPELINE_NAME,
                                         'StatisticsGen')
  stats_artifacts = stat_gen_output[standard_component_specs.STATISTICS_KEY]

  schema_gen_output = get_latest_artifacts(metadata_handler,
                                           SCHEMA_PIPELINE_NAME, 'SchemaGen')
  schema_artifacts = schema_gen_output[standard_component_specs.SCHEMA_KEY]

これで、各コンポーネントの出力を調べられるようになりました。前述のとおり、`StatisticsGen` と `SchemaGen` では [Tensorflow Data Validation(TFDV)](https://www.tensorflow.org/tfx/data_validation/get_started) が使用されており、TFDV はこれらのコンポーネントの出力を可視化することもできます。

このチュートリアルでは、TFDV を内部的に使用して可視化を表示する、TFX の可視化ヘルパーメソッドを使用します。

#### StatisticsGen の出力を調べる


In [ ]:
# docs-infra: no-execute
visualize_artifacts(stats_artifacts)

<!-- <img class="tfo-display-only-on-site"
src="images/penguin_tfdv/penguin_tfdv_statistics.png"/> -->

入力データの様々な統計を確認できます。これらの統計は、データの初期スキーマを自動的に構築するために `SchemaGen` に提供されます。


#### SchemaGen の出力を調べる


In [ ]:
visualize_artifacts(schema_artifacts)

このスキーマは、StatisticsGen の出力から自動的に推論されます。4 つの浮動小数点型特徴量と 1 つの INT 型特徴量があるのが確認できます。

### 今後使用するためにスキーマをエクスポートする

生成されたスキーマを確認し、改良する必要があります。スキーマの確認が済めば、それを機械学習モデルのトレーニングを行う後続のパイプラインで使用できるように永続化する必要があります。言い換えれば、実際のユースケースでは、スキーマファイルをバージョン管理システムに追加することをお勧めしますが、このチュートリアルでは、単純に、事前定義済みのファイルシステムパスにスキーマをコピーすることにします。


In [ ]:
import shutil

_schema_filename = 'schema.pbtxt'
SCHEMA_PATH = 'schema'

os.makedirs(SCHEMA_PATH, exist_ok=True)
_generated_path = os.path.join(schema_artifacts[0].uri, _schema_filename)

# Copy the 'schema.pbtxt' file from the artifact uri to a predefined path.
shutil.copy(_generated_path, SCHEMA_PATH)

スキーマファイルは、[Protocol Buffers テキスト形式](https://googleapis.dev/python/protobuf/latest/google/protobuf/text_format.html)と [TensorFlow Metadata Schema proto](https://github.com/tensorflow/metadata/blob/master/tensorflow_metadata/proto/v0/schema.proto) のインスタンスを使用します。

In [ ]:
print(f'Schema at {SCHEMA_PATH}-----')
!cat {SCHEMA_PATH}/*

スキーマの定義は必ず確認するようにし、必要に応じて編集してください。このチュートリアルでは、生成されたスキーマをそのまま使用することにします。


## 入力 Example を検証して ML モデルをトレーニングする

[単純な TFX パイプラインのチュートリアル](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple)で作成したパイプラインに戻ります。機械学習モデルをトレーニングして、モデルのトレーニングコードの記述に、生成したスキーマを使用します。

また、受信するデータセットをスキーマと照合し、異常値や欠損値を探す [ExampleValidator](https://www.tensorflow.org/tfx/guide/exampleval) コンポーネントも追加します。


### モデルトレーニングのコードを記述する

モデルのコードは、[単純な TFX パイプラインのチュートリアル](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple)で行ったように記述する必要があります。

モデル自体は前のチュートリアルと同じですが、今回は、手作業で特徴量を指定する代わりに、前のパイプラインから生成されたスキーマを使用します。ほとんどのコードに変更はありませんが、唯一、このファイルでは特徴の名前と型を指定する必要がないことが異なります。代わりに、名前と型を *schema* ファイルから読み取ります。

In [ ]:
_trainer_module_file = 'penguin_trainer.py'

In [ ]:
%%writefile {_trainer_module_file}

from typing import List
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_transform.tf_metadata import schema_utils

from tfx import v1 as tfx
from tfx_bsl.public import tfxio
from tensorflow_metadata.proto.v0 import schema_pb2

# We don't need to specify _FEATURE_KEYS and _FEATURE_SPEC any more.
# Those information can be read from the given schema file.

_LABEL_KEY = 'species'

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10

def _input_fn(file_pattern: List[str],
              data_accessor: tfx.components.DataAccessor,
              schema: schema_pb2.Schema,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    schema: schema of the input data.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      schema=schema).repeat()


def _build_keras_model(schema: schema_pb2.Schema) -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://www.tensorflow.org/guide/keras/overview for all API options.

  # ++ Changed code: Uses all features in the schema except the label.
  feature_keys = [f.name for f in schema.feature if f.name != _LABEL_KEY]
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in feature_keys]
  # ++ End of the changed code.

  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  # ++ Changed code: Reads in schema file passed to the Trainer component.
  schema = tfx.utils.parse_pbtxt_file(fn_args.schema_path, schema_pb2.Schema())
  # ++ End of the changed code.

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      schema,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      schema,
      batch_size=_EVAL_BATCH_SIZE)

  model = _build_keras_model(schema)
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  # The result of the training should be saved in `fn_args.serving_model_dir`
  # directory.
  model.save(fn_args.serving_model_dir, save_format='tf')

これで、モデルトレーニング用の TFX パイプラインを構築するすべての準備が完了しました。

### パイプライン定義を記述する

`Importer` と `ExampleValidator` という 2 つの新しいコンポーネントを追加します。Importer は TFX パイプラインに外部ファイルを取り込みます。この場合、スキーマ定義を含むファイルです。ExampleValidator は入力データを調べ、すべての入力データが、提供したデータスキーマに適合しているかどうかを検証します。


In [ ]:
def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     schema_path: str, module_file: str, serving_model_dir: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
  """Creates a pipeline using predefined schema with TFX."""
  # Brings data into the pipeline.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # Computes statistics over data for visualization and example validation.
  statistics_gen = tfx.components.StatisticsGen(
      examples=example_gen.outputs['examples'])

  # NEW: Import the schema.
  schema_importer = tfx.dsl.Importer(
      source_uri=schema_path,
      artifact_type=tfx.types.standard_artifacts.Schema).with_id(
          'schema_importer')

  # NEW: Performs anomaly detection based on statistics and data schema.
  example_validator = tfx.components.ExampleValidator(
      statistics=statistics_gen.outputs['statistics'],
      schema=schema_importer.outputs['result'])

  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],
      schema=schema_importer.outputs['result'],  # Pass the imported schema.
      train_args=tfx.proto.TrainArgs(num_steps=100),
      eval_args=tfx.proto.EvalArgs(num_steps=5))

  # Pushes the model to a filesystem destination.
  pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)))

  components = [
      example_gen,

      # NEW: Following three components were added to the pipeline.
      statistics_gen,
      schema_importer,
      example_validator,

      trainer,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

### パイプラインを実行する


In [ ]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      schema_path=SCHEMA_PATH,
      module_file=_trainer_module_file,
      serving_model_dir=SERVING_MODEL_DIR,
      metadata_path=METADATA_PATH))

パイプラインが正常に完了すると、「INFO:absl:Component Pusher is finished.」が表示されます。

### パイプラインの出力を調べる

ペンギンの分類モデルをトレーニングし、ExampleValidator で入力 Example も検証しました。前のパイプラインで行ったように、ExampleValidator の出力を分析できます。

In [ ]:
metadata_connection_config = tfx.orchestration.metadata.sqlite_metadata_connection_config(
    METADATA_PATH)

with Metadata(metadata_connection_config) as metadata_handler:
  ev_output = get_latest_artifacts(metadata_handler, PIPELINE_NAME,
                                   'ExampleValidator')
  anomalies_artifacts = ev_output[standard_component_specs.ANOMALIES_KEY]

ExampleValidator の ExampleAnomalies も可視化できます。

In [ ]:
visualize_artifacts(anomalies_artifacts)

Example の各分割に対し、「No anomalies found」が表示されます。このパイプラインでは、スキーマ生成に使用したのと同じデータを使用したため、期待される異常値はありません。このパイプラインを新しい受信データを使って繰り返し実行すれば、ExampleValidator によって、新しいデータと既存のスキーマ間の矛盾が見つかるでしょう。

以上が見つかった場合は、データを確認し、Example が自分の想定に従っているかどうかを確認するとよいでしょう。StatisticsGen などの他のコンポーネントの出力が役立つ場合がありますが、見つかった異常値によって以降のパイプライン実行がブロックされることはありません。

## 次のステップ

その他のリソースは、https://www.tensorflow.org/tfx/tutorials に掲載されています。

TFX の様々な概念についての詳細は、[TFX パイプラインの理解](https://www.tensorflow.org/tfx/guide/understanding_tfx_pipelines)をご覧ください。
